In [1]:

#%fs rm -r /FileStore/tables
##import os
#os.remove('/dbfs/FileStore/tables/PRED_30JUN17.csv')

In [2]:

#import libraries
#from pyspark.sql.functions import avg
import matplotlib.pyplot as plt
#plt.rcParams['font.size'] = 22
import warnings    # To suppress warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix


In [3]:
df1 = pd.read_csv('/dbfs/FileStore/tables/PRED_31MAR17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df2 = pd.read_csv('/dbfs/FileStore/tables/PRED_30JUN17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df3 = pd.read_csv('/dbfs/FileStore/tables/PRED_31OCT17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df4 = pd.read_csv('/dbfs/FileStore/tables/PRED_31DEC17.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df5 = pd.read_csv('/dbfs/FileStore/tables/PRED_31MAR18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df6 = pd.read_csv('/dbfs/FileStore/tables/PRED_30JUN18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df7 = pd.read_csv('/dbfs/FileStore/tables/PRED_31OCT18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df8 = pd.read_csv('/dbfs/FileStore/tables/PRED_31DEC18.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
df9 = pd.read_csv('/dbfs/FileStore/tables/PRED_MARR19.csv', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])


list_df = [df1,df2,df3,df4,df5,df6,df7,df8,df9]
for i in list_df:
  i = i.drop_duplicates().reset_index(drop = True)
  

In [4]:
'''
for i in list_df:
  i.replace(' ', np.nan, inplace=True)
  '''

Out[3]: "\nfor i in list_df:\n i.replace(' ', np.nan, inplace=True)\n "

In [5]:
temp1=df1[df1.EMP_STATUS=='A']
temp2=df2[df2.EMP_STATUS=='A']
temp3=df3[df3.EMP_STATUS=='A']
temp4=df4[df4.EMP_STATUS=='A']
temp5=df5[df5.EMP_STATUS=='A']
temp6=df6[df6.EMP_STATUS=='A']
temp7=df7[df7.EMP_STATUS=='A']
temp8=df8[df8.EMP_STATUS=='A']
temp9=df9[df9.EMP_STATUS=='A']



In [6]:
fd1=pd.merge(temp1[['MASK_EMPLOYEEID','EMP_STATUS']],df2,how='inner',on=['MASK_EMPLOYEEID'])
fd2=pd.merge(temp2[['MASK_EMPLOYEEID','EMP_STATUS']],df3,how='inner',on=['MASK_EMPLOYEEID'])
fd3=pd.merge(temp3[['MASK_EMPLOYEEID','EMP_STATUS']],df4,how='inner',on=['MASK_EMPLOYEEID'])
fd4=pd.merge(temp4[['MASK_EMPLOYEEID','EMP_STATUS']],df5,how='inner',on=['MASK_EMPLOYEEID'])
fd5=pd.merge(temp5[['MASK_EMPLOYEEID','EMP_STATUS']],df6,how='inner',on=['MASK_EMPLOYEEID'])
fd6=pd.merge(temp6[['MASK_EMPLOYEEID','EMP_STATUS']],df7,how='inner',on=['MASK_EMPLOYEEID'])
fd7=pd.merge(temp7[['MASK_EMPLOYEEID','EMP_STATUS']],df8,how='inner',on=['MASK_EMPLOYEEID'])
fd8=pd.merge(temp8[['MASK_EMPLOYEEID','EMP_STATUS']],df9,how='inner',on=['MASK_EMPLOYEEID'])

In [7]:


#Deletng EMP_STATUS column from original df
for i in range(8):
    data=globals()["fd" + str(i + 1)]
    data.drop(['EMP_STATUS_x'],axis=1,inplace=True)
    data.rename(columns={'EMP_STATUS_y':'TARGET_VALUE'},inplace=True)
    globals()["fd" + str(i + 1)]=data

In [8]:
training_set = pd.concat([fd1,fd2,fd3,fd4,fd5,fd6]).drop_duplicates().reset_index(drop=True)
validation_set = pd.concat([fd7,fd8]).drop_duplicates().reset_index(drop=True)

In [9]:
training_set.head()

,MASK_EMPLOYEEID,EMPLOYEE_AGE,GENDER,NO_OF_LEAVES,TOTAL_REPORTEES,BAND,DEPARTMENT_ID,NO_OF_MANAGER_REPORTEES,TECHM_EXPERIENCE,TOTAL_EXPERIENCE,LAST_HIKE_PERCENT,MANAGER_RATING,NO_OF_PREVIOUS_EMPLOYERS,TENURE_LAST_EMPLOPYER,HIGHEST_EDUCATION,QUARTILE,LAST_PROMOTION_DATE,TOTAL_PROMOTIONS,MANAGER_LAST_PROMOTION,LAST_ONSITE_TRAVEL_DATE,LAST_ONSITE_TRAVEL_DAYS,REWARDS,PRIMARY_SKILL,SECONDARY_SKILL,BASE_LOCATION_CITY,CURRENT_LOCATION_CITY,HIRE_LOCATION_CITY,RESIGNATION_FLG,BILLABILITY_STATUS,CURRENT_RATING,LAST_RATING,ALL_PREVIOUS_RAINGS,LATEST_HIKE_PERCENT,OFFICERCODE_DESC,GRIEVANCE_FLG,TARGET_VALUE,HIRE_EXIT_DATE
0,E006B574A10D5213,34.0,M,14,0,P1,IBU-CTP-01,22,5.1,12.5,0.0,NaN,2,4.92,BE/BTech/BScTech,3,NaT,0,2013-07-01,NaT,0,1,Business Analysis,NaN,BENGALURU,BENGALURU,BENGALURU,N,Y,NaN,C,"2013-E,2014-C,2015-C,2016-C",0.0,Lateral Joinees,NaN,A,2012-05-14
1,DE72B59D51C7B2AE,36.0,F,6,6,P1,IASE,74,12.3,12.3,0.0,NaN,0,0.00,DIPLOMA,0,2016-01-01,2,2010-07-01,NaT,0,3,Software Quality Assurance,NaN,HYDERABAD,HYDERABAD,HYDERABAD,N,Y,NaN,NaN,"2006-C,2007-E,2008-E,2009-E,2010-E,2011-C,2012...",0.0,Initial Training Program,N,A,2005-03-14
2,D0776ED5162671E1,46.0,M,5,0,P2,IASE,61,11.8,22.4,0.0,NaN,5,0.38,NaN,0,2007-07-01,1,NaT,2013-12-30,0,0,Project Management,NaN,HYDERABAD,HYDERABAD,HYDERABAD,N,Y,NaN,NaN,"2006-C,2007-C,2008-X,2009-C,2010-X,2011-C,2012...",0.0,Lateral Joinees,N,A,2005-08-26
3,5654CB68AE69AAD9,47.0,M,32,1,P2,IBU-ANZ3,30,10.4,20.9,0.0,NaN,3,8.44,Company Secretery,0,2011-07-01,1,NaT,2014-02-21,313,1,IBM-DB2-Performance Expert,NaN,HYDERABAD,HYDERABAD,HYDERABAD,N,Y,NaN,NaN,"2008-C,2009-C,2010-X,2011-C,2012-X,2013-C,2014...",0.0,FCTP,N,A,2007-02-13
4,2E8CB900573C1337,40.0,M,16,1,U4,IMS04C,34,10.1,14.3,0.0,NaN,2,3.04,NaN,0,2011-07-01,1,2010-07-01,2013-09-03,0,2,Application Design and Arch,NaN,HYDERABAD,HYDERABAD,HYDERABAD,N,Y,NaN,NaN,"2008-E,2009-E,2010-C,2011-C,2012-E,2013-E,2014...",0.0,Lateral Joinees,N,A,2007-06-01


In [10]:
f1 = df2.append(df3)
f1 = df2.append(df4)
triple1=pd.merge(temp1[['MASK_EMPLOYEEID','EMP_STATUS']],f1,how='inner',on=['MASK_EMPLOYEEID'])

f2 = df3.append(df4)
f2 = df3.append(df5)
triple2=pd.merge(temp2[['MASK_EMPLOYEEID','EMP_STATUS']],f2,how='inner',on=['MASK_EMPLOYEEID'])

f3 = df4.append(df5)
f3 = df4.append(df6)
triple3=pd.merge(temp3[['MASK_EMPLOYEEID','EMP_STATUS']],f3,how='inner',on=['MASK_EMPLOYEEID'])

f4 = df5.append(df6)
f4 = df5.append(df7)
triple4=pd.merge(temp4[['MASK_EMPLOYEEID','EMP_STATUS']],f4,how='inner',on=['MASK_EMPLOYEEID'])


f5 = df6.append(df7)
f5 = df6.append(df8)
triple5=pd.merge(temp5[['MASK_EMPLOYEEID','EMP_STATUS']],f5,how='inner',on=['MASK_EMPLOYEEID'])


list_f = [triple1,triple2,triple3,triple4,triple5]
for i in list_f:
  i['MASK_EMPLOYEEID'] = i['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)


#triples['MASK_EMPLOYEEID'].drop_duplicates().reset_index(drop = True)
  

In [11]:

#Deletng EMP_STATUS column from original df
for i in range(5):
    data=globals()["triple" + str(i + 1)]
    data.drop(['EMP_STATUS_x'],axis=1,inplace=True)
    data.rename(columns={'EMP_STATUS_y':'TARGET_VALUE'},inplace=True)
    globals()["triple" + str(i + 1)]=data

In [12]:
'''y1 = df2.append(df3)
y1 = df2.append(df4)
year1=pd.merge(temp1[['MASK_EMPLOYEEID','EMP_STATUS']],y1,how='inner',on=['MASK_EMPLOYEEID'])
year1['MASK_EMPLOYEEID'] = year1['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)
#year1 = year1.drop_duplicates(keep=False,inplace=True)

y2 = df6.append(df7)
y2 = df6.append(df8)
year2=pd.merge(temp5[['MASK_EMPLOYEEID','EMP_STATUS']],y2,how='inner',on=['MASK_EMPLOYEEID'])
year2['MASK_EMPLOYEEID'] = year2['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)
#year2 = year2.drop_duplicates(keep=False,inplace=True)

'''

Out[11]: "y1 = df2.append(df3)\ny1 = df2.append(df4)\nyear1=pd.merge(temp1[['MASK_EMPLOYEEID','EMP_STATUS']],y1,how='inner',on=['MASK_EMPLOYEEID'])\nyear1['MASK_EMPLOYEEID'] = year1['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)\n#year1 = year1.drop_duplicates(keep=False,inplace=True)\n\ny2 = df6.append(df7)\ny2 = df6.append(df8)\nyear2=pd.merge(temp5[['MASK_EMPLOYEEID','EMP_STATUS']],y2,how='inner',on=['MASK_EMPLOYEEID'])\nyear2['MASK_EMPLOYEEID'] = year2['MASK_EMPLOYEEID'].drop_duplicates( keep='last', inplace=False)\n#year2 = year2.drop_duplicates(keep=False,inplace=True)\n\n"

In [13]:
'''
#Deletng EMP_STATUS column from original df
for i in range(2):
    data=globals()["year" + str(i + 1)]
    data.drop(['EMP_STATUS_x'],axis=1,inplace=True)
    data.rename(columns={'EMP_STATUS_y':'TARGET_VALUE'},inplace=True)
    globals()["year" + str(i + 1)]=data'''

In [14]:
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
# print(fd1.QUARTILE.value_counts())
for i in llist:
  print('------------------')
  print(i.QUARTILE.value_counts())
  print((i[i.TARGET_VALUE=='I'].QUARTILE.value_counts()*100)/i.QUARTILE.value_counts())

In [15]:
labels=fd1.QUARTILE.value_counts().index
# colors=["olive","orange","hotpink","slateblue","y","lime"]
#explode=[0,0,0,0,0,0]
sizes=(fd1[fd1.TARGET_VALUE=='I'].QUARTILE.value_counts()*100)/fd1.QUARTILE.value_counts().values
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,autopct="%1.1f%%")
plt.title("Quartile Counts",color="saddlebrown",fontsize=15)
display()

In [16]:
labels=fd1.QUARTILE.value_counts().index
# colors=["olive","orange","hotpink","slateblue","y","lime"]
#explode=[0,0,0,0,0,0]
sizes=i[i.TOTAL_EXPERIENCE<=3].QUARTILE.value_counts().values
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,autopct="%1.1f%%")
plt.title("Education Field Counts",color="saddlebrown",fontsize=15)
display()

In [17]:
var1=pd.crosstab(data["QUARTILE"],data['TARGET_VALUE'])
var1.div(var1.sum(1).astype(float), axis=0).plot(kind="bar", stacked=False, figsize = (10,5)) 
  #plt.show()
display()

In [18]:
#total percentage of experienced and inexperienced employee in each quartile for each quarter
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
l=[0.0,1.0,2.0,3.0,4.0]
j=1
for i in llist:
  print('Quarter ' , str(j))
  print('total percent of inexperienced employee')
  print(i[i.TOTAL_EXPERIENCE<=3].QUARTILE.value_counts()*100/i.QUARTILE.value_counts())
  print('total percent of experienced employee')
  print(i[i.TOTAL_EXPERIENCE>3].QUARTILE.value_counts()*100/i.QUARTILE.value_counts())
  j+=1

In [19]:
#percentage of experienced and inexperienced people who are attriting


llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
l=[0.0,1.0,2.0,3.0,4.0]
j=1
for i in llist:
  print('Quarter ' , str(j))
  print('total percent of inexperienced employee')
  print(i[i.TOTAL_EXPERIENCE<=3][i.TARGET_VALUE=='I'].QUARTILE.value_counts()*100/i[i.TOTAL_EXPERIENCE<=3].QUARTILE.value_counts())
  print('total percent of experienced employee')
  print(i[i.TOTAL_EXPERIENCE>3][i.TARGET_VALUE=='I'].QUARTILE.value_counts()*100/i[i.TOTAL_EXPERIENCE>3].QUARTILE.value_counts())
  print('-----------')
  j+=1

In [20]:

#tech mahindra experience per quartile

llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
l=[0.0,1.0,2.0,3.0,4.0]
j=1
for i in llist:
  print(i[i.TECHM_EXPERIENCE<=2].QUARTILE.value_counts()*100/i.QUARTILE.value_counts())

In [21]:

# (fd1[fd1.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts()>3).sum(),fd1[fd1.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts()
# fd1[fd1.TARGET_VALUE=='I'][fd1.TOTAL_EXPERIENCE>2].TOTAL_PROMOTIONS.value_counts(),fd1[fd1.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.mean()

j=1
for i in llist:
    print('QUARTER NUMBER ',str(j))
    print('the percentage of total inexperienced and experienced',i[i.TOTAL_EXPERIENCE<=2].TOTAL_EXPERIENCE.value_counts().sum()*100/i.TOTAL_EXPERIENCE.value_counts().sum(),i[i.TOTAL_EXPERIENCE>2].TOTAL_EXPERIENCE.value_counts().sum()*100/i.TOTAL_EXPERIENCE.value_counts().sum())
    print('percent of the inexperienced who are leaving',i[i.TOTAL_EXPERIENCE<=3][i.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts().sum()*100/i[i.TOTAL_EXPERIENCE<=3].TOTAL_EXPERIENCE.value_counts().sum())
    print('percent of the experienced who are leaving',i[i.TOTAL_EXPERIENCE<=3][i.TARGET_VALUE=='I'].TOTAL_EXPERIENCE.value_counts().sum()*100/i[i.TOTAL_EXPERIENCE>3].TOTAL_EXPERIENCE.value_counts().sum())
    print('-------------------')
    j+=1

In [22]:

j=1
for i in llist:
  print('for QUARTER ',str(j))
  print((i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE<=3].TOTAL_PROMOTIONS.value_counts()*100)/i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE<=3].TOTAL_PROMOTIONS.value_counts().sum())
  print((i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE>3].TOTAL_PROMOTIONS.value_counts()*100)/i[i.TARGET_VALUE=='I'][i.TOTAL_EXPERIENCE>3].TOTAL_PROMOTIONS.value_counts().sum())
  print('-------------------------')
  j+=1

In [23]:
def stripplott(variable):
  f,ax=plt.subplots(figsize=(18,18))
  sb.set(rc={'figure.figsize':(16,20)})
  sb.stripplot(x='TARGET_VALUE', y=variable, data=fd1, alpha=0.3, jitter=True);
  # plt.plot()
  display()
  stripplott("TOTAL_EXPERIENCE")

In [24]:
f,ax=plt.subplots(figsize=(18,18))
sb.set(rc={'figure.figsize':(100,50)})
# sb.countplot(x='BAND',data=fd3)
sb.factorplot(x='TARGET_VALUE',col='TOTAL_PROMOTIONS',kind='count',data=fd1)
# fd1.groupby('NO_OF_LEAVES').BAND.hist(alpha=0.6)
# fig, ax = plt.subplots(1, 2, figsize=(300, 100))
# sb.kdeplot(fd1.loc[fd1['TARGET_VALUE'] == 'I', 'NO_OF_MANAGER_REPORTEES'], ax=ax[0], label='INACTIVE(0)')
# sb.kdeplot(fd1.loc[fd1['TARGET_VALUE'] == 'A', 'NO_OF_MANAGER_REPORTEES'], ax=ax[0], label='ACTIVE(1)')
# plt.show()
display()

#  Model Building

In [26]:
#retain original copy

data = fd8[fd8.TARGET_VALUE=='I'].copy()
data1 = training_set.copy()

In [27]:
from scipy.stats import chisquare, chi2_contingency, chi2

# One-way chisquare test to see if states are evenly distributed
chisquare(df['GRIEVANCE_FLG'].value_counts())

#Contingency Table
contingency_table=pd.crosstab(df["HIGHEST_EDUCATION"],df["TARGET_VALUE"])
print('contingency_table :-\n',contingency_table)

#Observed Values
Observed_Values = contingency_table.values 
print("Observed Values :-\n",Observed_Values)

#Expected Values
import scipy.stats
b=scipy.stats.chi2_contingency(contingency_table)
Expected_Values = b[3]
print("Expected Values :-\n",Expected_Values)


In [28]:
#Degree of Freedom
no_of_rows=len(contingency_table.iloc[0:2,0])
no_of_columns=len(contingency_table.iloc[0,0:2])
df=(no_of_rows-1)*(no_of_columns-1)
print("Degree of Freedom:-",df)

#or
#df=b[2]
#print("Degree of Freedom:-",df)

In [29]:
#Significance Level 5%
alpha=0.05
#chi-square statistic - χ2
from scipy.stats import chi2
chi_square=sum([(o-e)**2./e for o,e in zip(Observed_Values,Expected_Values)])
chi_square_statistic=chi_square[0]+chi_square[1]
print("chi-square statistic:-",chi_square_statistic)

#critical_value
critical_value=chi2.ppf(q=1-alpha,df=df)
print('critical_value:',critical_value)

#p-value
p_value=1-chi2.cdf(x=chi_square_statistic,df=df)
print('p-value:',p_value)
#p-value: 0.7641771556220945
print('Significance level: ',alpha)
print('Degree of Freedom: ',df)
print('chi-square statistic:',chi_square_statistic)
print('critical_value:',critical_value)
print('p-value:',p_value)

In [30]:
#compare chi_square_statistic with critical_value and p-value which is the probability of getting chi-square>0.09 (chi_square_statistic)
if chi_square_statistic>=critical_value:
    print("Reject H0,There is a relationship between 2 categorical variables")
else:
    print("Retain H0,There is no relationship between 2 categorical variables")
    
if p_value<=alpha:
    print("Reject H0,There is a relationship between 2 categorical variables")
else:
    print("Retain H0,There is no relationship between 2 categorical variables")

In [31]:
cat_df = data1.select_dtypes(include=['object']).copy()
cat_df.head()

In [32]:

#fd1['PROMOTION_FLAG']=fd1.apply(lambda fd1 : 1 if fd1.LAST_PROMOTION_DATE.year>2016 and fd1.LAST_PROMOTION_DATE.month<6 else 0,axis=1)
data1['PROMOTION_FLAG']=data1.apply(lambda data1 : 1 if data1.LAST_PROMOTION_DATE.year>2016 and data1.LAST_PROMOTION_DATE.month<6 else 0,axis=1)

In [33]:
#data2=fd1.copy()
#data2.info()
data1['CITY_FLAG']=data1.apply(lambda data1 : 1 if data1.HIRE_LOCATION_CITY==data1.CURRENT_LOCATION_CITY else 0,axis=1 )
data1.CITY_FLAG.value_counts()

In [34]:
data1.PRIMARY_SKILL.fillna(data1.SECONDARY_SKILL.str[0],inplace=True)

In [35]:
data1.CURRENT_RATING.fillna(data1.LAST_RATING,inplace=True)



In [36]:
data1.CURRENT_RATING.fillna(data1.ALL_PREVIOUS_RAINGS.str[-1],inplace=True)


In [37]:
import pickle

pickle_in = open("band.pickle","rb")
dict_band = pickle.load(pickle_in)
print(dict_band)

pickle_in = open("education.pickle","rb")
dict_education = pickle.load(pickle_in)
print(dict_education)

pickle_in = open("rating.pickle","rb")
dict_rating = pickle.load(pickle_in)
print(dict_rating)

pickle_in = open("education_name.pickle","rb")
dict_edu_name = pickle.load(pickle_in)
print(dict_edu_name)
#print(example_dict[3])

In [38]:
data1['HIGHEST_EDUCATION'].replace(dict_edu_name, inplace=True)

In [39]:
data1['BAND'].replace(dict_band, inplace=True)
data1['CURRENT_RATING'].replace(dict_rating, inplace=True)
data1['LAST_RATING'].replace(dict_rating, inplace=True)
data1['MANAGER_RATING'].replace(dict_rating, inplace=True)
data1['HIGHEST_EDUCATION'].replace(dict_education, inplace=True)
   

In [40]:

print(data1.dtypes)

In [41]:
#Conversion of object to category 
for i in data1:
    if data1[i].dtype==object:
        data1[i] = data1[i].astype('category').cat.codes


In [42]:
llisst = ['EMPLOYEE_AGE','NO_OF_LEAVES','TOTAL_REPORTEES','NO_OF_MANAGER_REPORTEES','TECHM_EXPERIENCE','TOTAL_EXPERIENCE','NO_OF_PREVIOUS_EMPLOYERS','LAST_ONSITE_TRAVEL_DAYS','TOTAL_PROMOTIONS','MANAGER_RATING','CURRENT_RATING','LAST_RATING','REWARDS','QUARTILE','TENURE_LAST_EMPLOPYER','LATEST_HIKE_PERCENT','LAST_HIKE_PERCENT','HIGHEST_EDUCATION','CITY_FLAG','Month']
for j in llisst:
      data1[j] = data1[j].fillna(0.0).astype(int)


In [44]:
data1.dtypes

In [45]:

corr = data1.corr()
f,ax=plt.subplots(figsize=(10,10))
#b.heatmap(corr,annot=True,linewidths=0.5,linecolor="green",fmt=".1f",ax=ax)
sb.heatmap(corr,annot=False,linewidths=0.5,linecolor="green",fmt=".1f",ax=ax)
#plt.show()
display()


In [46]:
'''#data1 = pd.get_dummies(data1, columns=['carrier'], prefix = ['carrier'])
data1 = pd.get_dummies(data1, columns=['GENDER','DEPARTMENT_ID','PRIMARY_SKILL','SECONDARY_SKILL','CURRENT_LOCATION_CITY','BASE_LOCATION_CITY','HIRE_LOCATION_CITY','RESIGNATION_FLG','BILLABILITY_STATUS','ALL_PREVIOUS_RAINGS','OFFICERCODE_DESC','GRIEVANCE_FLG'], prefix = ['GENDER','DEPARTMENT_ID','PRIMARY_SKILL','SECONDARY_SKILL','CURRENT_LOCATION_CITY','BASE_LOCATION_CITY','HIRE_LOCATION_CITY','RESIGNATION_FLG','BILLABILITY_STATUS','ALL_PREVIOUS_RAINGS','OFFICERCODE_DESC','GRIEVANCE_FLG'])

print(data1.head())
'''

In [47]:

dft = data1.pop('HIRE_EXIT_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['HIRE_EXIT_DATE']=dft


dft = data1.pop('LAST_PROMOTION_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['LAST_PROMOTION_DATE']=dft

dft = data1.pop('MANAGER_LAST_PROMOTION') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['MANAGER_LAST_PROMOTION']=dft


dft = data1.pop('LAST_ONSITE_TRAVEL_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['LAST_ONSITE_TRAVEL_DATE']=dft



In [48]:

#to put target variable in last
dft = data1.pop('TARGET_VALUE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['TARGET_VALUE']=dft # add b series as a 'new' column.
#df['x']=df2 # add b series as a 'new' column.


In [49]:
data1.dtypes

In [50]:
data1.shape

In [51]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [52]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [53]:
data1.columns

In [54]:


features = data1.iloc[:,0:35]
target = data1.iloc[:,-1]
#splitting the dataset into training and testing



In [55]:
features.info

In [56]:
# load the dataset
#dataframe = pandas.read_csv('airline-passengers.csv', usecols=[1], engine='python')
dataset = features.values
dataset = dataset.astype('float32')

In [57]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [58]:

#to put target variable in last
dft = data1.pop('TARGET_VALUE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['TARGET_VALUE']=dft # add b series as a 'new' column.
#df['x']=df2 # add b series as a 'new' column.


In [59]:

dft = data1.pop('HIRE_EXIT_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['HIRE_EXIT_DATE']=dft


dft = data1.pop('LAST_PROMOTION_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['LAST_PROMOTION_DATE']=dft

dft = data1.pop('MANAGER_LAST_PROMOTION') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['MANAGER_LAST_PROMOTION']=dft


dft = data1.pop('LAST_ONSITE_TRAVEL_DATE') # remove column b and store it in df1
#df2 = df.pop('x') # remove column x and store it in df2
data1['LAST_ONSITE_TRAVEL_DATE']=dft



In [60]:

# split into train and test sets
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,0:34], dataset[train_size:len(dataset),:-1]
print(len(train), len(test))

In [61]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [62]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [63]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 2, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 2, testX.shape[1]))

In [64]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=2, batch_size=32, verbose=2)

In [65]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [66]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [67]:
feature_train,feature_test,target_train, target_test  = train_test_split(features,target,test_size=0.2,random_state=42)
sc = StandardScaler()
feature_train = sc.fit_transform(feature_train)
feature_test = sc.transform(feature_test)


In [68]:
# Scaling the training set
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [69]:
 Since LSTMs store long term memory state, we create a data structure with 60 timesteps and 1 output
# So for each element of training set, we have 60 previous training set elements 
X_train = []
y_train = []
for i in range(60,2769):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)


In [70]:
#Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Third LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Fourth LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='rmsprop',loss='mean_squared_error')
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=50,batch_size=32)

In [71]:


#features = data.iloc[:,[1,3,4,7,8,9,10,12,13,16,17,18,29]]
#target = data.iloc[:,]
features = data1.iloc[:,0:34]
target = data1.iloc[:,-1]
#splitting the dataset into training and testing



In [72]:
 features.info()

In [73]:
'''We will use three methods for feature selection:

Remove collinear features
Remove features with greater than a threshold percentage of missing values
Keep only the most relevant features using feature importances from a model'''
#Identify Correlated Variables
# Threshold for removing correlated variables
threshold = 90

# Absolute value correlation matrix
corr_matrix = (features.corr().abs())*100
corr_matrix.head(30)

In [74]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

In [75]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))
print(to_drop)

In [76]:
features = features.drop(['TOTAL_EXPERIENCE', 'CURRENT_LOCATION_CITY', 'HIRE_LOCATION_CITY'], axis=1)
#target = target
#df = pd.DataFrame.drop(['Meter ID', 'abc'], axis=1)

print('Training shape: ', features.shape)
print('Testing shape: ', target.shape)

In [77]:
import tensorflow as  tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

In [78]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error

In [79]:
# Some functions to help out with
def plot_predictions(test,predicted):
    plt.plot(test, color='red',label='Real ')
    plt.plot(predicted, color='blue',label='Predicted ')
    plt.title('Attrition Prediction')
    plt.xlabel('Time')
    plt.ylabel('Attrition')
    plt.legend()
    plt.show()

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

In [80]:
'''# First, we get the data
dataset = pd.read_csv('../input/IBM_2006-01-01_to_2018-01-01.csv', index_col='Date', parse_dates=['Date'])
dataset.head()
'''
d1 = pd.read_csv('/dbfs/FileStore/tables/PRED_31MAR17.csv',index_col='HIRE_EXIT_DATE', parse_dates = ['HIRE_EXIT_DATE','LAST_PROMOTION_DATE','MANAGER_LAST_PROMOTION','LAST_ONSITE_TRAVEL_DATE'])
d1.head()

In [81]:
# Checking for missing values
training_set = dataset[:'2016'].iloc[:,1:2].values
test_set = dataset['2017':].iloc[:,1:2].values
# We have chosen 'High' attribute for prices. Let's see what it looks like
dataset["High"][:'2016'].plot(figsize=(16,4),legend=True)
dataset["High"]['2017':].plot(figsize=(16,4),legend=True)
plt.legend(['Training set (Before 2017)','Test set (2017 and beyond)'])
plt.title('IBM stock price')
plt.show()

In [82]:
# network architecture
model = Sequential()
model.add(LSTM(50, input_shape=(feature_train.shape[1], feature_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

# fit
history = model.fit(feature_train, target_train, epochs=1, batch_size=10, validation_data=(feature_test, target_test), verbose=2, shuffle=False)

In [83]:
# bulid model
model = Sequential()
model.add(LSTM(128, input_shape=(feature_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))


In [84]:
#model.save('Minst_Lstm.model')

In [85]:

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
random_forest = RandomForestClassifier(n_estimators = 800, criterion = 'entropy', random_state = 0)
random_forest.fit(feature_train, target_train)

def plot_feature_importances(importances, features):
    # get the importance rating of each feature and sort it
    indices = np.argsort(importances)

    # make a plot with the feature importance
    plt.figure(figsize=(12,14), dpi= 80, facecolor='w', edgecolor='k')
    plt.grid()
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], height=0.8, color='mediumvioletred', align='center')
    plt.axvline(x=0.03)
    plt.yticks(range(len(indices)), list(features))
    plt.xlabel('Relative Importance')
    plt.show()
    display()

plot_feature_importances(random_forest.feature_importances_, features)


In [86]:

#Logistic regression
classifier = LogisticRegression(random_state= 0)
classifier.fit(feature_train,target_train)
pred = classifier.predict(feature_test)
print(pred)
cm = confusion_matrix(target_test,pred)
print(cm)
score = classifier.score(feature_test,target_test)
print(score)


In [87]:

#KNN
from sklearn.neighbors import KNeighborsClassifier
classs =  KNeighborsClassifier(n_neighbors = 5,p = 2)
classs.fit(feature_train,target_train)
pred1  = classs.predict(feature_test)
score1 = classs.score(feature_test,target_test)
print(score1)

In [88]:

'''



# Exploratory Data Analysis - Numerical Features
#Let's visualize data distribution of numerical features. Usually, prediction model works well if the data distribution is normal distribution. So, if there are skewed data distirubtion then we can make them normal distribution through log transformations.

#without log transformation
fig = plt.figure(figsize=(16,4))

# Histogram Plot for Employee Age
plt.subplot(1,2,1)
plt.hist(emp_viz_df['EMPLOYEE_AGE'], bins=15, color='green')
plt.title('Distribution of Employee Age')
plt.xlabel("Employee Age")

# Histogram Plot for employee Monthly Income
plt.subplot(1,2,2)
plt.hist(emp_viz_df['TOTAL_PROMOTIONS'], bins=30, color='red')
plt.title('Distribution of Employee Monthly Income')
plt.xlabel("Employee Monthly Income")

fig.tight_layout();
display()










#with log 
fig = plt.figure(figsize=(16,4))

# Histogram Plot for Employee Age
plt.subplot(1,2,1)
plt.hist(np.log1p(emp_viz_df['EMPLOYEE_AGE']), bins=20, color='green')
plt.title('Distribution of Employee Age')
plt.xlabel("Log of Employee Age")

# Histogram Plot for employee Monthly Income
plt.subplot(1,2,2)
plt.hist(np.log1p(emp_viz_df['TOTAL_PROMOTIONS']), bins=15, color='red')
plt.title('Distribution of Employee Monthly Income')
plt.xlabel("Log of Employee Monthly Income")

fig.tight_layout();
display()







print('Skewness in employee Age feature: ', emp_viz_df['EMPLOYEE_AGE'].skew())
print('Skewness in employee Monthly Income feature: ', emp_viz_df['TOTAL_PROMOTIONS'].skew())
"""
"""We can observe above that without log transformation, "Age" feature looks normally distributed more than "MonthlyIncome" feature.
Hence, after log transformation, there is hardly any change in "Age" distributions but there is significant transformation in "MonthlyIncome" feature.
This can also be confirmed by checking the skewness of each feature by applying numpy skew metric and higher the skewness, higher the value.
So, we will use some threshold value of skewness to carry out log transformation on features with high skewness."""

In [89]:

# Basic statistics of categorical features
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  print(i.describe(include=[np.object]))
#cat_col_names = data.select_dtypes(include=[np.object]).columns.tolist() # Get categorical feature names
#cat_col_names

In [90]:
#Attrition Target Variable Distribution
lllist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in lllist:
  attrition_freq = i[['TARGET_VALUE']].apply(lambda x: x.value_counts())
  attrition_freq['frequency_percent'] = round((100 * attrition_freq / attrition_freq.sum()),2)

  print(attrition_freq)


In [91]:
def distribution(variable):
  for i in llist:
      print(i[variable].value_counts(),(i[i.TARGET_VALUE=='I'][variable].value_counts()*100)/i[variable].value_counts(),i[i.TARGET_VALUE=='I'][variable].value_counts())

In [92]:
distribution('DEPARTMENT_ID')

In [93]:
llist = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  print(distribution('OFFICERCODE_DESC'))

In [94]:
 """To understand how each features are impacting the attrition indicator, we need to first handle the null / missing values, otherwise our observations might not be accurate and will lead to wrong conclusions.
"""
for i in llist:
    null_feat_df = pd.DataFrame()
    null_feat_df['Null Count'] = i.isnull().sum().sort_values(ascending=False)
    null_feat_df['Null Pct'] = null_feat_df['Null Count'] / float(len(i))

    null_feat_df = null_feat_df[null_feat_df['Null Pct'] > 0]


    total_null_feats = null_feat_df.shape[0]
    null_feat_names = null_feat_df.index
    print('Total number of features having null values: ', total_null_feats)
    del null_feat_df

In [95]:
#filling with 0
for i in llist:
    i["LATEST_HIKE_PERCENT"].fillna("0", inplace = True) 
    i["LAST_HIKE_PERCENT"].fillna("0", inplace = True)

    i['EMPLOYEE_AGE'].fillna(i['EMPLOYEE_AGE'].mode()[0], inplace=True) 
#EMPLOYEE_AGE 

In [96]:

plt.figure(1) 
plt.subplot(221) 
data['GENDER'].value_counts(normalize=True).plot.bar(figsize=(20,10), title= 'Gender') 
plt.subplot(222) 
data['BAND'].value_counts(normalize=True).plot.bar(title= 'Band') 
plt.subplot(223) 
data['REWARDS'].value_counts(normalize=True).plot.bar(title= 'REWARDS') 
plt.subplot(224) 
#plt.show()
display()

In [97]:
# Segregating data set based on Attrition value

#Attrition across Job Roles
fig, ax6 = plt.subplots(1,2, figsize=(24,10))
attr_yes = data[data.TARGET_VALUE=='I'] #subset with Attrition = Yes
attr_no = data[data.TARGET_VALUE=='A'] ##subset with Attrition = No
sb.countplot(x='BAND', hue='GENDER', data = attr_yes, palette="Set3", ax = ax6[1])
sb.countplot(x='BILLABILITY_STATUS', hue='GENDER', data = attr_yes,palette="Set3", ax = ax6[0])
plt.show()
display()
    

In [98]:
#
def plot_count(feature, title,size=1):
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(data))
    g = sb.countplot(data[feature], order = data[feature].value_counts(normalize=True).index[:20], palette='Set3')
    g.set_title("Number and percentage of {}".format(title))
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show()   
    display()

In [99]:
plot_count('CURRENT_RATING','current rating')

In [100]:
#
plt.figure(figsize=(15, 8))
cols = ['LAST_ONSITE_TRAVEL_DAYS', 'REWARDS', 'TOTAL_PROMOTIONS', 'TENURE_LAST_EMPLOPYER', 'NO_OF_PREVIOUS_EMPLOYERS']
uniques = [len(data[col].unique()) for col in cols]
sb.set(font_scale=1.2)
sb.color_palette("Blues")
ax = sb.barplot(cols, uniques,log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 
# for col, uniq in zip(cols, uniques):
#     ax.text(col, uniq, uniq, color='black', ha="center")
display()

In [101]:
def divisionplot(variable):
  var=pd.crosstab(data[variable],data['QUARTILE'])
  var.div(var.sum(1).astype(float), axis=0).plot(kind="bar", stacked=False, figsize = (10,4)) 
  #plt.show()
  display()

In [102]:

divisionplot('BAND')

In [103]:

sb.jointplot(x="TOTAL_PROMOTIONS", y="TOTAL_EXPERIENCE", data=data,  ratio=3, color="r")
plt.show()
display()



In [104]:
#main_data = fd4.copy()
listdf = [fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in listdf:
  print(i['CURRENT_RATING'].value_counts() , i['CURRENT_RATING'][i['TARGET_VALUE'] == 'I'].value_counts())
  #print(i['CURRENT_RATING'][i['TARGET_VALUE'] == 'I'].value_counts())
  

In [105]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['CURRENT_RATING']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['CURRENT_RATING']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)


In [106]:
data['AGE_Interval'] = pd.cut(data['EMPLOYEE_AGE'], 6, labels=['19-25','25-35', '35-45', '45-55', '55-65','65+'])

In [107]:

age=pd.DataFrame(data.groupby("AGE_Interval")[["TOTAL_REPORTEES","NO_OF_LEAVES","NO_OF_MANAGER_REPORTEES","TECHM_EXPERIENCE","TOTAL_EXPERIENCE","LAST_HIKE_PERCENT","NO_OF_PREVIOUS_EMPLOYERS","TENURE_LAST_EMPLOPYER","TOTAL_PROMOTIONS","LAST_ONSITE_TRAVEL_DAYS","REWARDS","LATEST_HIKE_PERCENT","HIGHEST_EDUCATION"]].mean())
age["Count"]=data.AGE_Interval.value_counts(dropna=False)
age.reset_index(level=0, inplace=True)
age.head()

In [108]:
plt.figure(figsize=(6,6))
ax=sb.barplot(x=age.AGE_Interval,y=age.Count)
plt.xticks(rotation=90)
plt.xlabel("Age")
plt.ylabel("Counts")
plt.title("Age Counts")
plt.show()
display()

In [109]:

data['AGE_Interval'].value_counts()

In [110]:
plt.figure(figsize=(6,6))
ax=sb.barplot(x=age.AGE_Interval,y=age.TOTAL_PROMOTIONS,palette = sb.cubehelix_palette(len(age.index)))
plt.xticks(rotation=90)
plt.xlabel("Age")
plt.ylabel("TOTAL_PROMOTIONS")
plt.title("TOTAL_PROMOTIONS According to Age")
plt.show()
display()

In [111]:
"""
"""
labels=data.AGE_Interval.value_counts().index
# colors=["olive","orange","hotpink","slateblue","y","lime"]
#explode=[0,0,0,0,0,0]
sizes=(data[data.TARGET_VALUE=='I'].AGE_Interval.value_counts()*100)/data.AGE_Interval.value_counts().values
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,autopct="%1.1f%%")
plt.title("AGE interval Counts",color="saddlebrown",fontsize=15)
display()


In [112]:
age1=pd.DataFrame(data.groupby("EMPLOYEE_AGE")[["TOTAL_REPORTEES","NO_OF_LEAVES","NO_OF_MANAGER_REPORTEES","TECHM_EXPERIENCE","TOTAL_EXPERIENCE","LAST_HIKE_PERCENT","NO_OF_PREVIOUS_EMPLOYERS","TENURE_LAST_EMPLOPYER","TOTAL_PROMOTIONS","LAST_ONSITE_TRAVEL_DAYS","REWARDS","LATEST_HIKE_PERCENT","HIGHEST_EDUCATION"]].mean())
age1["Count"]=data.EMPLOYEE_AGE.value_counts(dropna=False)
age1.reset_index(level=0, inplace=True)
age1.head()

In [113]:
agenorm=age1.apply(lambda x: x/max(x))
agenorm.head()

In [114]:
ageb=data.EMPLOYEE_AGE.value_counts().index
f,ax=plt.subplots(figsize=(15,15))
sb.pointplot(y=agenorm.LAST_HIKE_PERCENT,x=ageb,color="purple",alpha=0.8)
sb.pointplot(y=agenorm.TOTAL_PROMOTIONS,x=ageb,color="sandybrown",alpha=0.8)
plt.text(5,0.65,"LAST_HIKE_PERCENT",color="purple",fontsize=15,style="italic")
plt.text(5,0.63,"TOTAL_PROMOTIONS",color="sandybrown",fontsize=15,style="italic")
plt.xlabel("Age",fontsize=15,color="darkred")
plt.ylabel("Values",fontsize=15,color="darkred")
plt.title("LAST_HIKE_PERCENTe VS JTOTAL_PROMOTIONS",fontsize=15,color="darkred")
plt.grid()
display()

In [115]:
ageb=data.EMPLOYEE_AGE.value_counts().index
f,ax=plt.subplots(figsize=(15,15))
sb.pointplot(y=agenorm.LATEST_HIKE_PERCENT,x=ageb,color="purple",alpha=0.8)
sb.pointplot(y=agenorm.TOTAL_PROMOTIONS,x=ageb,color="sandybrown",alpha=0.8)
plt.text(5,0.65,"LATEST_HIKE_PERCENT",color="purple",fontsize=15,style="italic")
plt.text(5,0.63,"TOTAL_PROMOTIONS",color="sandybrown",fontsize=15,style="italic")
plt.xlabel("Age",fontsize=15,color="darkred")
plt.ylabel("Values",fontsize=15,color="darkred")
plt.title("LAST_HIKE_PERCENTe VS JTOTAL_PROMOTIONS",fontsize=15,color="darkred")
plt.grid()
display()

In [116]:
f,ax = plt.subplots(figsize = (15,10))
sb.kdeplot(agenorm.REWARDS,agenorm.LAST_ONSITE_TRAVEL_DAYS,shade=True,cut=1)
display()

In [117]:
f,ax = plt.subplots(figsize = (15,10))
sb.kdeplot(agenorm.TOTAL_PROMOTIONS,agenorm.LAST_ONSITE_TRAVEL_DAYS,shade=True,cut=1)
display()

In [118]:
promotions=pd.DataFrame(data.groupby("BAND").TOTAL_PROMOTIONS.mean().sort_values(ascending=False))
plt.figure(figsize=(5,5))
ax=sb.barplot(x=promotions.index,y=promotions.TOTAL_PROMOTIONS)
plt.xticks(rotation=90)
plt.xlabel("BAND")
plt.ylabel("TOTAL_PROMOTIONS")
plt.title("BAND with TOTAL_PROMOTIONS")
plt.show()
display()

In [119]:
band=pd.DataFrame(data.groupby("BAND")["EMPLOYEE_AGE","NO_OF_LEAVES","NO_OF_PREVIOUS_EMPLOYERS","NO_OF_MANAGER_REPORTEES","LAST_ONSITE_TRAVEL_DAYS","LATEST_HIKE_PERCENT","LAST_HIKE_PERCENT","NO_OF_MANAGER_REPORTEES","TENURE_LAST_EMPLOPYER","TECHM_EXPERIENCE","TOTAL_EXPERIENCE"].mean())
band

In [120]:
f,ax = plt.subplots(figsize = (9,10))
sb.barplot(x=band.LATEST_HIKE_PERCENT,y=band.index,color='green',alpha = 0.5,label='LATEST_HIKE_PERCENT' )
sb.barplot(x=band.LAST_ONSITE_TRAVEL_DAYS,y=band.index,color='blue',alpha = 0.7,label='LAST_ONSITE_TRAVEL_DAYS')
sb.barplot(x=band.TECHM_EXPERIENCE,y=band.index,color='cyan',alpha = 0.6,label='Years At Company')
sb.barplot(x=band.NO_OF_PREVIOUS_EMPLOYERS,y=band.index,color='yellow',alpha = 0.6,label='NO_OF_PREVIOUS_EMPLOYERS')
sb.barplot(x=band.NO_OF_LEAVES,y=band.index,color='red',alpha = 0.6,label='NO_OF_LEAVES')

ax.legend(loc='right',frameon = True)     
ax.set(xlabel='Values', ylabel='band',title = "band with Different Features")
plt.show()
display()

In [121]:
g = sb.pairplot(data, vars=["TOTAL_PROMOTIONS", "REWARDS"],hue="BAND",size=5)
display()

In [122]:

f,ax = plt.subplots(figsize = (5,5))
sb.boxplot(x="GENDER",y="EMPLOYEE_AGE",hue="QUARTILE",data=data,palette="Paired")
display()

In [123]:
f,ax = plt.subplots(figsize = (15,10))
ax.legend(frameon=False, loc='lower center', ncol=2)

sb.boxplot(x="GENDER",y="EMPLOYEE_AGE",hue="BAND" ,data=data,palette="hls")
display()

In [124]:

f,ax = plt.subplots(figsize = (5,5))
sb.pointplot(x="GENDER", y="TECHM_EXPERIENCE", hue="TARGET_VALUE", data=data,
              palette={"I": "blue", "A": "pink"},
              markers=["*", "o"], linestyles=["-", "--"]);
display()
## Inference
# Males and Females within the ranges for 'TotalWorkingYears' of 11 to 13 are less likely to quit

In [125]:
data['TECHM_Interval'] = pd.cut(data['TECHM_EXPERIENCE'], 6, labels=['0-5','5-10', '10-15', '15-20', '20-25','25+'])

In [126]:
f,ax = plt.subplots(figsize = (5,5))
#plt.subplots(figsize=(15,5))
sb.countplot(data.TECHM_Interval)
display()

In [127]:
data['HIGHEST_EDUCATION'].value_counts()


#print(data['HIGHEST_EDUCATION'].value_counts())

In [128]:
data1

In [129]:
data1['EDUCATION_CATEGORIES'].value_counts()

In [130]:
f,ax = plt.subplots(figsize = (11,5))
#plt.subplots(figsize=(15,5))
sb.countplot(data.EDUCATION_CATEGORIES)
display()

In [131]:
labels=data.EDUCATION_CATEGORIES.value_counts().index
colors=["olive","orange","hotpink","slateblue","y","lime","b"]
explode=[1,0,0,0,1,1]
sizes=data.EDUCATION_CATEGORIES.value_counts() 
plt.figure(figsize=(7,7))
plt.pie(sizes,labels=labels,colors=colors,autopct="%1.1f%%")
plt.title("Education Field Counts",color="saddlebrown",fontsize=15)
display()


In [132]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  i['LeavesInterval'] = pd.cut(i['NO_OF_LEAVES'][i['NO_OF_LEAVES']<50], 5, labels=['<10', '<20', '<30', '<40','<50+'])
  freq = i[['LeavesInterval']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['LeavesInterval']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)

In [133]:
#data['LeavesInterval'].value_counts()
data['LeavesInterval'] = pd.cut(i['NO_OF_LEAVES'][i['NO_OF_LEAVES']<50], 5, labels=['<10', '<20', '<30', '<40','<50+'])

In [134]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['DEPARTMENT_ID']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['DEPARTMENT_ID']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  #print(freq)
  print(freq["frequency_percent"].max())

In [135]:
# library and dataset
#import seaborn as sns
#df = sns.load_dataset('iris')
f,ax = plt.subplots(figsize = (15,10))
# plot of 2 variables
sb.kdeplot(data['TOTAL_REPORTEES'], shade=True, color="r")
sb.kdeplot(data['NO_OF_MANAGER_REPORTEES'], shade=True, color="b")
#sns.plt.show()
display()

In [136]:
# library and dataset
#import seaborn as sns
#df = sns.load_dataset('iris')
f,ax = plt.subplots(figsize = (5,5))
# plot of 2 variables
sb.kdeplot(data['TOTAL_PROMOTIONS'], shade=True, color="r")
sb.kdeplot(data['REWARDS'], shade=True, color="b")
#sns.plt.show()
display()

In [137]:
data['HIRE_LOCATION_CITY'].value_counts()

In [138]:
idx1 = pd.Index(data.CURRENT_LOCATION_CITY)
idx2 = pd.Index(data.BASE_LOCATION_CITY)
idx3 = pd.Index(data.HIRE_LOCATION_CITY)
#main_data['unique'] = idx1.difference(idx2).values
print(idx1.difference(idx2).values)
print(idx2.difference(idx1).values)
print(idx3.difference(idx1).values)
#print(idx3.difference(idx2).values)
#array([3, 6])

In [139]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['HIRE_LOCATION_CITY']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['HIRE_LOCATION_CITY']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)

In [140]:
#LAST_RATING,CURRENT_RATING
for i in listdf:
  freq = i[['HIRE_EXIT_DATE']].apply(lambda x: x.value_counts())
  attr_yes_rating =i[['HIRE_EXIT_DATE']][i['TARGET_VALUE'] == 'I'].apply(lambda x: x.value_counts()) 
  freq['frequency_percent'] = round((100 * attr_yes_rating / freq.sum()),2)
  #display()
  print(freq)
  
  
  

In [141]:
data.columns

In [142]:
features=['LeavesInterval','TECHM_Interval','CURRENT_RATING','AGE_Interval', 'TOTAL_PROMOTIONS','DEPARTMENT_ID ','QUARTILE']
fig=plt.subplots(figsize=(10,15))
for i, j in enumerate(features):
    plt.subplot(4, 2, i+1)
    plt.subplots_adjust(hspace = 1.0)
    sb.countplot(x=j,data = data)
    plt.xticks(rotation=90)
    plt.title("No. of employee")
    display()

In [143]:
# pie chart of workers
labels = ['M', 'F']
sizes = [data['GENDER'].value_counts()[0],
         data['GENDER'].value_counts()[1]
        ]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True)
ax1.axis('equal')
plt.title('Gender Pie Chart', fontsize=20)
plt.show()
display()

## Inference
# Proportion of male is higher than female associates

In [144]:
fig=plt.subplots(figsize=(5,5))
sb.factorplot(data=data,y='TOTAL_PROMOTIONS',x='BAND',size=7,aspect=2,kind='point')
display()


In [145]:
x

In [146]:
"""
# Plotting the KDEplots
f, axes = plt.subplots(3, 3, figsize=(10, 8), 
                       sharex=False, sharey=False)

# Defining our colormap scheme
s = np.linspace(0, 3, 7)
cmap = sb.cubehelix_palette(start=0.0, light=1, as_cmap=True)

# Generate and plot
x = data['EMPLOYEE_AGE'].values
y = data['TECHM_EXPERIENCE'].values
sb.kdeplot(x, y, cmap=cmap, shade=True, cut=5, ax=axes[0,0])
axes[0,0].set( title = 'Age Vs TechMExperience')
#display()
cmap = sb.cubehelix_palette(start=0.333333333333, light=1, as_cmap=True)

# Generate and plot
x = data['EMPLOYEE_AGE'].values
y = data['NO_OF_LEAVES'].values
sb.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[0,1])
axes[0,1].set( title = 'Age Vs no_of_leaves')
#display()

cmap = sb.cubehelix_palette(start=0.666666666667, light=1, as_cmap=True)
# Generate and plot
x = data['NO_OF_PREVIOUS_EMPLOYERS'].values
y = data['EMPLOYEE_AGE'].values
sb.kdeplot(x, y, cmap=cmap, shade=True, ax=axes[0,2])
axes[0,2].set( title = 'No_of_prev_employers Vs Age')
display()
'''
cmap = sns.cubehelix_palette(start=1.0, light=1, as_cmap=True)
# Generate and plot
x = attrition['DailyRate'].values
y = attrition['DistanceFromHome'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[1,0])
axes[1,0].set( title = 'Daily Rate against DistancefromHome')

cmap = sns.cubehelix_palette(start=1.333333333333, light=1, as_cmap=True)
# Generate and plot
x = attrition['DailyRate'].values
y = attrition['JobSatisfaction'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[1,1])
axes[1,1].set( title = 'Daily Rate against Job satisfaction')

cmap = sns.cubehelix_palette(start=1.666666666667, light=1, as_cmap=True)
# Generate and plot
x = attrition['YearsAtCompany'].values
y = attrition['JobSatisfaction'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[1,2])
axes[1,2].set( title = 'Daily Rate against distance')

cmap = sns.cubehelix_palette(start=2.0, light=1, as_cmap=True)
# Generate and plot
x = attrition['YearsAtCompany'].values
y = attrition['DailyRate'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[2,0])
axes[2,0].set( title = 'Years at company against Daily Rate')

cmap = sns.cubehelix_palette(start=2.333333333333, light=1, as_cmap=True)
# Generate and plot
x = attrition['RelationshipSatisfaction'].values
y = attrition['YearsWithCurrManager'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[2,1])
axes[2,1].set( title = 'Relationship Satisfaction vs years with manager')

cmap = sns.cubehelix_palette(start=2.666666666667, light=1, as_cmap=True)
# Generate and plot
x = attrition['WorkLifeBalance'].values
y = attrition['JobSatisfaction'].values
sns.kdeplot(x, y, cmap=cmap, shade=True,  ax=axes[2,2])
axes[2,2].set( title = 'WorklifeBalance against Satisfaction')
f.tight_layout()
dispaly()
'''
"""

In [147]:
"""
train_small = data# train.sample(frac=0.2).copy() # not small for now
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
sb.kdeplot(train_small.loc[data['BILLABILITY_STATUS'] == 'N', 'EMPLOYEE_AGE'], ax=ax[0], label='NoPay(0)')
sb.kdeplot(train_small.loc[data['BILLABILITY_STATUS'] == 'Y', 'EMPLOYEE_AGE'], ax=ax[0], label='HasPay(1)')

#\train_small.loc[data['HasDetections'] == 0, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
#train_small.loc[train['HasDetections'] == 1, 'DefaultBrowsersIdentifier'].hist(ax=ax[1])
#ax[1].legend(['NoDetection(0)', 'HasDetection(1)'])

plt.show()
display()
"""
'''
'''

In [148]:
"""
'''
# Segregating data set based on Attrition value
attr_yes = data[data.TARGET_VALUE=='I'] #subset with Attrition = Yes
attr_no = data[data.TARGET_VALUE=='A'] ##subset with Attrition = No
'''
attrition = data[(data['TARGET_VALUE'] == 'A' )]
no_attrition = data[(data['TARGET_VALUE'] == 'I')]
'''
# Segregating data set based on Attrition value
attr_yes = data[data.TARGET_VALUE=='I'] #subset with Attrition = Yes
attr_no = data[data.TARGET_VALUE=='A'] ##subset with Attrition = No
#Add columns to store discrete variables for Salary range and alcohol levels
#Adding discrete valriables
data['LeavesInterval'] = pd.cut(data['NO_OF_LEAVES'], 5, labels=['<9', '<15', '<24', '<32', '<33+'])
data['TechMExpLvl'] = pd.cut(data['TECHM_EXPERIENCE'], 5, labels=['lvl1', 'lvl2', 'lvl3', 'lvl4', 'lvl5'])
data['traveldate'] = pd.cut(data['LAST_ONSITE_TRAVEL_DAYS'], 5, labels=['0to3', '3to6', '6to9', '9to12', '12+'])



#Factor Plot
sb.factorplot(x =   'TARGET_VALUE',     # Categorical
               y =   'NO_OF_LEAVES',          # Continuous
               hue = 'DEPARTMENT_ID',   # Categorical
               col = 'LeavesInterval',   # Categorical for graph columns
               col_wrap=3,           # Wrap facet after two axes
               kind = 'box',
               data = data)
plt.show()
display()


# Attrition count across Daily Rate, Years in company, Years since last promotion and level of stock options
fig, ax2 = plt.subplots(2,2, figsize=(10,10))
sb.countplot(x='LeavesInterval', data=attr_yes, ax = ax2[0,0])
sb.countplot(x='TechMExpLvl', data=attr_yes, ax = ax2[0,1])
sb.countplot(x='traveldate', data=attr_yes, ax = ax2[1,0])
#sb.countplot(x='StockOptionLevel', data=attr_yes, ax = ax2[1,1])
plt.show()
display()
'''
"""
"""
#Count Plot
# Attrition across Education level and corresponding percentage across the total number of employees
total = data.shape[0] 
hrfig = sb.countplot(x='HIGHEST_EDUCATION', hue = 'TARGET_VALUE', data = data)
#Above graph showcases the percentages of the employees across Education levels and corresponding attri
for p in hrfig.patches:
    height = p.get_height()
    hrfig.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format(height*100/total),
            ha="center") 
plt.show()
display()
"""


In [149]:
'''from datetime import date
from datetime import datetime

dt = datetime.combine(date.today(), datetime.min.time())
print(dt)'''

In [150]:
"""
# Attrition distribution bar plot

plot = attrition_freq[['frequency_percent']].plot(kind="bar");
plot.set_title("Attrition Distribution", fontsize=40);
plot.grid(color='lightgray', alpha=0.5);
display()
"""
"""Just by a quick inspection of the counts of the number of 'Yes' and 'No' in the target variable tells us that there is quite a large skew in target variable.
Therefore we have to keep in mind that there is quite a big imbalance in our target variable. Many statistical techniques have been put forth to treat imbalances in data (oversampling or undersampling).
In this notebook, I will use an oversampling technique known as SMOTE to treat this imbalance.
We will see the prediction model with and without SMOTE treatment for imbalance class issue."""


In [151]:
"""
#Step 4 - Feature Engineering
Feature engineering is one aspect which provided a huge impact on the outcome rather than the model. Here, we try at creating new features with the existing variables we have based on my assumptions.

Step 4.1 - Addition of New Features
Tenure per job: Usually, people who have worked with many companies but for small periods at every organization tend to leave early as they always need a change of Organization to keep them going.
Years without Change: For any person, a change either in role or job level or responsibility is needed to keep the work exciting to continue. We create a variable to see how many years it has been for an employee without any sort of change using Promotion, Role and Job Change as a metric to cover different variants of change.
Compensation Ratio: Compa Ratio is the ratio of the actual pay of an Employee to the midpoint of a salary range. The salary range can be that of his/her department or organization or role. The benchmark numbers can be a organization’s pay or Industry average.


"""
emp_proc_df = data.copy() # Copy cleaned dataset for feature engineering

emp_proc_df['TenurePerJob'] = 0

for i in range(0, len(emp_proc_df)):
    if emp_proc_df.loc[i,'NO_OF_PREVIOUS_EMPLOYERS'] > 0:
        emp_proc_df.loc[i,'TenurePerJob'] = emp_proc_df.loc[i,'TOTAL_EXPERIENCE'] / emp_proc_df.loc[i,'NO_OF_PREVIOUS_EMPLOYERS']
"""
emp_proc_df['YearWithoutChange1'] = emp_proc_df['YearsInCurrentRole'] - emp_proc_df['YearsSinceLastPromotion']
emp_proc_df['YearWithoutChange2'] = emp_proc_df['TotalWorkingYears'] - emp_proc_df['YearsSinceLastPromotion']

monthly_income_median = np.median(emp_proc_df['MonthlyIncome'])
emp_proc_df['CompRatioOverall'] = emp_proc_df['MonthlyIncome'] / monthly_income_median

print('Dataset dimension: {} rows, {} columns'.format(emp_proc_df.shape[0], emp_proc_df.shape[1]))
"""









full_col_names = emp_proc_df.columns.tolist()
num_col_names = emp_proc_df.select_dtypes(include=[np.int64,np.float64]).columns.tolist() # Get numerical feature names

# Preparing list of ordered categorical features
for i in emp_proc_df:
    if emp_proc_df[i].dtype==object:
        num_cat_col_names = emp_proc_df[i]
#num_cat_col_names = ['DEPARTMENT_ID', 'TARGET_VALUE', 'OFFICERCODE_DESC', 'ALL_PREVIOUS_RAINGS', 'JobSatisfaction',
                     #'PerformanceRating', 'RelationshipSatisfaction', 'WorkLifeBalance', 'StockOptionLevel']

target = ['TARGET_VALUE']

num_col_names = list(set(num_col_names) - set(num_cat_col_names)) # Numerical features w/o Ordered Categorical features
cat_col_names = list(set(full_col_names) - set(num_col_names) - set(target)) # Categorical & Ordered Categorical features

print('Total number of numerical features: ', len(num_col_names))
print('Total number of categorical & ordered categorical features: ', len(cat_col_names))
cat_emp_df = emp_proc_df[cat_col_names]
num_emp_df = emp_proc_df[num_col_names]









#Transform Numerical Features
#In order to fix the skewness, let’s take the log for all numerical features with an absolute skew greater than 80% (Note: log+1, to avoid division by zero issues).

# Let's create dummy variables for each categorical attribute for training our calssification model
for col in num_col_names:
    if num_emp_df[col].skew() > 0.80:
        num_emp_df[col] = np.log1p(num_emp_df[col])

num_emp_df.head()





""" #Transform Categorical Features
Machine Learning model works only on numerical datasets, hence, we need to transform categorical features into numerical features.
One of the best strategy is to convert each category value into a new column and assigns 1 or 0 (True/False) value to the column. This has the benefit of not weighting a value improperly but does have the downside of adding more columns to the data set.
This approach is also called as "One Hot Encoding". We can use Pandas feature get_dummies to achieve this transformation.
There is another way to handled ordered categorical feature is to give ordered value based on their definitions i.e Low-Meidum-High would be 1-2-3. We can try this approach some other time. But, this can be evaluated to check the performance of the model.
"""
# Let's create dummy variables for each categorical attribute for training our calssification model
for col in cat_col_names:
    col_dummies = pd.get_dummies(cat_emp_df[col], prefix=col)
    cat_emp_df = pd.concat([cat_emp_df, col_dummies], axis=1)

# Use the pandas apply method to numerically encode our attrition target variable
attrition_target = emp_proc_df['TARGET_VALUE'].map({'I':1, 'A':0})

# Drop categorical feature for which dummy variables have been created

cat_emp_df.drop(cat_col_names, axis=1, inplace=True)
cat_emp_df.head()




# Correlation of Numerical Features against Attrition
num_corr_df = num_emp_df[['EMPLOYEE_AGE', 'NO_OF_MANAGER_REPORTEES', 'TOTAL_REPORTEES']]
corr_df = pd.concat([num_corr_df, attrition_target], axis=1)
corr = corr_df.corr()
display()
plt.figure(figsize = (10, 8))
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sb.axes_style("white")
#sns.heatmap(data=corr, annot=True, mask=mask, square=True, linewidths=.5, vmin=-1, vmax=1, cmap="YlGnBu")
sb.heatmap(data=corr, annot=True, square=True, linewidths=.5, vmin=-1, vmax=1, cmap="YlGnBu")
plt.show()
display()






# Another way to check for correlation between attributes is to use Pandas’ scatter_matrix function,

from pandas.plotting import scatter_matrix

scatter_matrix(corr_df, figsize=(16, 10));
display()



# Concat the two dataframes together columnwise
final_emp_df = pd.concat([num_emp_df, cat_emp_df], axis=1)

print('Dataset dimension after treating categorical features with dummy variables: {} rows, {} columns'.format(final_emp_df.shape[0], final_emp_df.shape[1]))
final_emp_df.head()










"""
#Model Building and Validation
Since, we have to predict a binary class, we will be using classification models for training & predicting Employee Attrition. We need to keep in mind that our focus should be to have a better accuracy of predicting attrition i.e. Attrition = 1 which in confusion matrix will be "True Positive". However, we should not forget the prediction accuracy of not qualifying for attrition i.e. Attrition = 0 which will be "True Negative" in confusion matrix.

So, we need to focus on four parameters:

Accuracy: Overall, how often is the classifier correct? i.e {(TP+TN)/Total}
True Positive Rate: When it's actually yes, how often does it predict yes? default_ind = 1, {TP/Actual YES}, this is also known as "Sensitivity" or "Recall"
Precision: When it predicts yes, how often is it correct? i.e. {TP/(TP+FP)}
Specificity: When it's actually no, how often does it predict no? default_ind = 0, {TN/actual NO}
Cross Validation Score: Cross Validation is a technique which involves reserving a particular sample of a dataset on which you do not train the model. Later, you test your model on this sample before finalizing it. Do this for k folds and take mean of accuracy scores of the k fold models.
F1 Score: This is a weighted average of the true positive rate (recall) and precision.
ROC Curve: This is a commonly used graph that summarizes the performance of a classifier over all possible thresholds. It is generated by plotting the True Positive Rate (y-axis) against the False Positive Rate (x-axis) as you vary the threshold for assigning observations to a given class.
Above information was taken and more details can be found at https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

Step 5.1 - Prepare Train & Test Dataset
The data should be divided into train and test data. We will use train_test_split feature to divide the data and we will be using 70-30 ratio
"""
# Import the train_test_split method
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# Split data into train and test sets as well as for validation and testing
X_train, X_val, y_train, y_val = train_test_split(final_emp_df, attrition_target,
                                                  test_size= 0.30, random_state=42);

print("Stratified Sampling: ", len(X_train), "train set +", len(X_val), "validation set")

# Stratified Splitting
#split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
#for train_index, test_index in split.split(emp_data_proc, emp_data_proc['Gender']):
#    strat_train_set = emp_data_proc.loc[train_index]
#    strat_test_set = emp_data_proc.loc[test_index]





















from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
def gen_model_performance(actual_target, pred_target):
    model_conf_matrix = confusion_matrix(actual_target, pred_target)
    model_roc_score = roc_auc_score(actual_target, pred_target)
    model_accuracy = accuracy_score(actual_target, pred_target) * 100.0
    
    TP = model_conf_matrix[1][1]; TN = model_conf_matrix[0][0]; 
    FP = model_conf_matrix[0][1]; FN = model_conf_matrix[1][0];
    sensitivity = TP / float(TP + FN) * 100.0; specificity = TN / float(TN + FP) * 100.0;
    precision = TP / float(TP + FP) * 100.0;
    
    return sensitivity, specificity, model_accuracy, precision, model_roc_score
def evaluate_model_score(X, y, scoring='accuracy'):
    
    logreg_model = LogisticRegression(random_state=0)
    logreg_cv_model = LogisticRegressionCV()
    rfc_model = RandomForestClassifier()
    extrees_model = ExtraTreesClassifier()
    gboost_model = GradientBoostingClassifier()
    dt_model = DecisionTreeClassifier()
    aboost_model = AdaBoostClassifier()
    gnb_model = GaussianNB()

    models = [logreg_model, logreg_cv_model, dt_model, rfc_model, 
              extrees_model, gboost_model, aboost_model, gnb_model]
    
    model_results = pd.DataFrame(columns = ["Model", "Accuracy", "Precision", "CV Score",
                                            "Sensitivity","Specificity","ROC Score"])
    
    for model in models:
        model.fit(X, y,)
        y_pred = model.predict(X)
        score = cross_val_score(model, X, y, cv=5, scoring=scoring)
        
        sensitivity, specificity, accuracy, precision, roc_score = gen_model_performance(y, y_pred)
    
        scores = cross_val_score(model, X, y, cv=5)
    
        model_results = model_results.append({"Model": model.__class__.__name__,
                              "Accuracy": accuracy, "Precision": precision,
                              "CV Score": scores.mean()*100.0,
                              "Sensitivity": sensitivity, "Specificity": specificity,
                              "ROC Score": roc_score}, ignore_index=True)
    return model_results
 """

In [152]:
fd1.info()

In [153]:
llist=[fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  i.CURRENT_RATING.fillna(i.LAST_RATING,inplace=True)
  i.CURRENT_RATING.fillna(i.ALL_PREVIOUS_RAINGS.str[-1],inplace=True)

In [154]:
for i in range(fd1[fd1.CURRENT_RATING.isnull()].ALL_PREVIOUS_RAING.shape[0]):
  a=fd1[fd1.CURRENT_RATING.isnull()].ALL_PREVIOUS_RAING[i]
  print(a[-1])

In [155]:
llist=[fd1,fd2,fd3,fd4,fd5,fd6,fd7,fd8]
for i in llist:
  i.PRIMARY_SKILL.fillna(i.SECONDARY_SKILL,inplace=True)

In [156]:
'''s='20170601'
date=pd.to_datetime(s)
date'''

In [157]:

fd1['PROMOTION_FLAG']=fd1.apply(lambda fd1 : 1 if fd1.LAST_PROMOTION_DATE.year>2016 and fd1.LAST_PROMOTION_DATE.month<6 else 0,axis=1)
fd2['PROMOTION_FLAG']=fd2.apply(lambda fd2 : 1 if fd2.LAST_PROMOTION_DATE.year>2016 and fd2.LAST_PROMOTION_DATE.month<10 else 0,axis=1)
fd3['PROMOTION_FLAG']=fd3.apply(lambda fd3 : 1 if fd3.LAST_PROMOTION_DATE.year>2016 and fd3.LAST_PROMOTION_DATE.month<12 else 0,axis=1)
fd4['PROMOTION_FLAG']=fd4.apply(lambda fd4 : 1 if fd4.LAST_PROMOTION_DATE.year>2017 and fd4.LAST_PROMOTION_DATE.month<3 else 0,axis=1)
fd5['PROMOTION_FLAG']=fd5.apply(lambda fd5 : 1 if fd5.LAST_PROMOTION_DATE.year>2017 and fd5.LAST_PROMOTION_DATE.month<6 else 0,axis=1)
fd6['PROMOTION_FLAG']=fd6.apply(lambda fd6 : 1 if fd6.LAST_PROMOTION_DATE.year>2017 and fd6.LAST_PROMOTION_DATE.month<10 else 0,axis=1)
fd7['PROMOTION_FLAG']=fd7.apply(lambda fd7 : 1 if fd7.LAST_PROMOTION_DATE.year>2017 and fd7.LAST_PROMOTION_DATE.month<12 else 0,axis=1)
fd8['PROMOTION_FLAG']=fd8.apply(lambda fd8 : 1 if fd8.LAST_PROMOTION_DATE.year>2018 and fd8.LAST_PROMOTION_DATE.month<3 else 0,axis=1)